# Homework 1: CS344 (Ian Park)

## Exercise 1.1

**Q.** *The earliest psychologists studied the human mind using introspection. Would this be a good way for researchers to gain insight into human cognitive processes for the purpose of modelling those processes? (Please give a one-to-two-paragraph answer.)*

**A.** Introspection at best is limited by the biological capabilities of humans at a given moment. Even with a huge prefrontal cortex, it is hard for most people to even grasp large problems for a prolonged time. To make it worse, evolutionary human instincts can differ in each individual. Thus, if one were to model the human cognitive processes based soley on introspection, there will be many misleading results. Rather, it would be more reasonable to take an external look at the biological processes of human thinking. If there was enough data, however, introspection could give us some insight.

## Exercise 1.2


**Q.** *Describe your state and action representations.*

**A.**
 - state: cities
 - action: pairs of two cities

In [23]:
"""
Calvin University CS344 Homework01 Exercise02

This implements a local-search version for Travelling Sales Person.
It utilizes AIMA's hill-climbing and simulated-annealing search algorithms.

@author: Ian Park
@version: 27feb2020
"""

from aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
import time


class TSP(Problem):
    """An implementation of Travelling Sales Person for local search.
    State representation:
        [s1, s2, ..., sn]: State of locations in current path configuration.
    Action representation:
        [(a, b), (b, c), ..., (y, z)]: Possible places that can be visited.
    """

    def __init__(self, initial, costs):
        self.initial = initial
        self.costs = costs

    def actions(self, state):
        actions = []

        # Get all swappable actions (excluding initial and final)
        for i in range(1, len(state) - 1):
            for j in range(1, len(state) - 1):
                if i != j:
                    actions.append((i, j))

        return actions

    def result(self, state, action):
        new_state = state[:]
        # Swap chosen element and successor
        i, j = new_state[action[0]], new_state[action[1]]
        new_state[action[0]], new_state[action[1]] = j, i
        return new_state

    def value(self, state):
        value = 0
        for i in range(len(state) - 1):
            location1 = state[i]
            location2 = state[i + 1]

            # Check for costs in both directions
            if (location1, location2) in self.costs:
                value += self.costs[(location1, location2)]
            else:
                value += self.costs[(location1, location2)]
        return -value
    
from random import shuffle
# List of Locations
locations = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']

# Randomized(shuffled) initial state
initial = locations[:]
shuffle(initial)

# Keys: Paths from one point to another represented as tuples
# Values: Path Costs
costs = {}

# Generate randomized path costs
for i in range(len(locations)):
    for j in range(len(locations)):
        if i != j and (locations[i], locations[j]) not in costs:
            costs[(locations[i], locations[j])] = randrange(1, 11)

problem = TSP(initial, costs)

start = time.time()
hill_climbing = hill_climbing(problem)
end = time.time()

print('Hill climbing: ' + str(hill_climbing)
      + '\n\tvalue: ' + str(-problem.value(hill_climbing))
      + '\n\ttime: ' + str(end - start)
      )

start = time.time()
sim_annealing = simulated_annealing(problem, exp_schedule(k=20, lam=0.005, limit=1000))
end = time.time()

print('Simulated annealing: ' + str(sim_annealing)
      + '\n\tvalue: ' + str(-problem.value(sim_annealing))
      + '\n\ttime: ' + str(end - start)
      )

Hill climbing: ['b', 'j', 'd', 'k', 'g', 'a', 'h', 'i', 'e', 'f', 'c']
	value: 28
	time: 0.002141237258911133
Simulated annealing: ['b', 'g', 'k', 'j', 'e', 'f', 'h', 'i', 'd', 'a', 'c']
	value: 21
	time: 0.16291308403015137


**Q.** *Explain which local search algorithm works better and why.*

**A.** Simulated-annealing gave better results in most cases. However, hill-climbing was faster, which makes sense because it just lands on the closest local maximum and simulated-annealing needs time to cool down. I would say that if the problem was larger with more local maximums, simulated-annealing would be better because it could search for more possibilities.

## Exercise 1.3

In [24]:
"""
Calvin University CS344 Homework01 Exercise03

This implements a problem for class scheduling.
It utilizes AIMA's constraint search algorithms.
Only seems to work with backtracking.
Referenced problems from csp.py

@author: Ian Park
@version: 27feb2020
"""

from time import time
from aima.csp import CSP, AC3, backtracking_search, parse_neighbors
from aima.search import depth_first_graph_search

# Define a course scheduling formulation.
def Scheduling():
    variables = 'cs108 cs112 cs212 cs214 cs232 cs262 cs344'.split()
    profs = 'schuurman adams vanderlinden plantinga norman'.split()
    times = 'mwf800 mwf900 mwf1030 tth1130 tth1230'.split()
    rooms = 'nh253 sb382'.split()

    domain = {}
    # Find the domain through all the combinations of variables/profs/times/rooms
    for variable in variables:
        domain[variable] = []
        for prof in profs:
            for time in times:
                for room in rooms:
                    domain[variable].append(prof + ' ' + time + ' ' + room)

    # Dictionary of neighbors
    # creates a combination of the list of all classes without that key variable
    neighbors = {}
    for variable in variables:
        var_list = []
        for v in variables:
            if v != variable:
                var_list.append(v)
        neighbors[variable] = var_list

    def scheduling_constraint(A, a, B, b, recurse=0):
        # Split string values of each domain value
        part1 = str(a).split()
        part2 = str(b).split()

        # Check for prof and time conflicts
        if part1[0] == part2[0] and part1[1] == part2[1]:
            return False

        # Check for time and room conflicts
        if part1[1] == part2[1] and part1[2] == part2[2]:
            return False

        if recurse == 0:
            return scheduling_constraint(B, b, A, a, 1)

        return True

    return CSP(variables, domain, neighbors, scheduling_constraint)

# Print solution and time
def print_solution(schedule, start_time, end_time):
    for course in schedule:
        print(course, schedule[course])
    print("\ntime: " + str(end_time - start_time))
    
# Assign the problem
problem = Scheduling()

start = time()
# result = depth_first_graph_search(problem)
# result = AC3(problem)

# This seems like the only think to work with my implementation
result = backtracking_search(problem)

# result = min_conflicts(problem, max_steps=100000)
end= time()


if problem.goal_test(problem.infer_assignment()):
    print("solution:")
    print_solution(result, start, end)
else:
    print("\nfailed...")
    print(problem.curr_domains)
    problem.display(problem.infer_assignment())


solution:
cs108 schuurman mwf800 nh253
cs112 schuurman mwf900 nh253
cs212 schuurman mwf1030 nh253
cs214 schuurman tth1130 nh253
cs232 schuurman tth1230 nh253
cs262 adams mwf800 sb382
cs344 adams mwf900 sb382

time: 0.003413677215576172
